In [1]:
import json
import jsonlines
import os
import multiprocessing as mp
import subprocess, shlex
import simplejson as json
import pandas as pd

from dashboard.data_load import load_mouse_blastp_results, load_esmfold
from dashboard.etl import CACHE_DIR, PROTEIN_TOOLS_PATH, sorf_query

from Bio.Seq import Seq
from pathlib import Path
from tqdm import tqdm

from veliadb import base
from veliadb.base import Orf



2023-12-06 02:59:48.258 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:59:56.873 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:59:56.874 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:59:56.876 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:59:56.876 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:59:56.877 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:59:56.878 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:59:56.878 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-

In [2]:
session = base.Session()

In [3]:
import boto3
import gzip
from io import BytesIO
from Bio import SeqIO

# Initialize a boto3 client for S3
s3_client = boto3.client('s3')
# S3 Bucket and file details
bucket_name = 'velia-annotation-dev'
s3_file_key = 'genomes/hg38/GRCh38.p13.genome.fa.gz'
# Get the file object from S3
file_obj = s3_client.get_object(Bucket=bucket_name, Key=s3_file_key)
# Read the file content
fasta_gzipped = file_obj['Body'].read()
# Decompress the file
with gzip.open(BytesIO(fasta_gzipped), 'rt') as f:
    # Parse the FASTA file
    genome_reference = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [27]:
with open('all_vtx_gencode_231205.txt', 'r') as infile:
    ids = [int(i.replace('VTX-', '')) for i in infile.readlines()]


In [28]:
len(ids)

13529

In [6]:
orfs = session.query(Orf).filter(Orf.id.in_(ids)).all()


In [7]:
o = session.query(Orf).filter(Orf.id == 852124).one()


In [7]:
o.start

-1

In [8]:
x = sorf_query.parallel_sorf_query(852124, genome_reference)

In [29]:
def pfunc(i):
    r = sorf_query.parallel_sorf_query(i, genome_reference)
    return r

with open('sorf_table.jsonlines', 'w') as fopen:
    with mp.Pool(16) as ppool:
        for r in tqdm(ppool.imap(pfunc, ids), total=len(ids)):
            fopen.write(json.dumps(r))
            fopen.write('\n')

100%|██████████| 13529/13529 [42:22<00:00,  5.32it/s] 


In [30]:
sorf_df = sorf_query.load_jsonlines_table('sorf_table.jsonlines', index_col='vtx_id')


  0%|          | 0/13529 [00:00<?, ?it/s]

100%|██████████| 13529/13529 [00:00<00:00, 55460.22it/s]


In [31]:
for i, row in sorf_df.iterrows():
    assert len(row.aa) > 1
    assert '*' not in row.aa
    assert row.aa == Seq(row.nucl).translate().strip('*')

/opt/conda/envs/veliadash_updated/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [32]:
row.aa

'MVAALTWSMATSATASQARQGLIVKLILMTVQVTLVSMESVWMALIATVVSAHQDSQGRDVTLTLMSVPPIPVARVQHVSTV'

In [25]:
Seq(row.nucl).translate()

Seq('IPYFLQEVYLSMMIYKWLLPAQNPNFQDLCFLFLI')

In [26]:
assert x['aa'] == Seq(x['nucl']).translate()

NameError: name 'x' is not defined